### Old version 1

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

dir = 'D:/GD_awekimm/[YU]/[Project]/[Quantum]/Quantum_2nd/04_Analysis/QuanTech_R2/R file/'

# 1. 데이터 로딩 (기존과 동일)
df = pd.read_csv(dir+'quant_author_ed_eu_val.csv')
inst = pd.read_csv(dir+'quant_inst_ed_eu_val_cleaned.csv')

required_columns = [
    'pubid', 'city', 'country', 'pubyear', 'author_id_te', 
    'full_name', 'matched', 'organization_cleaned', 'suborganization_cleaned'
]
df = df[required_columns]
inst = inst[['pubid', 'organization_cleaned', 'suborganization_cleaned']]

df = df.drop_duplicates()
df['full_name'] = df['full_name'].str.replace(r'\.$', '', regex=True)

# 2. '정답' 그룹과 '처리 대상' 그룹 분리 (기존과 동일)
df_matched = df[df['matched'] == 'matched'].copy()
df_rest = df[df['matched'] == 'rest'].copy()

# =================================================================
# ⭐️ 3. (신규) 'rest' 데이터 소속 기관 추론 로직 ⭐️
# =================================================================

# 3-1. (준비) 추론에 필요한 정보 생성
# 'matched' 데이터에서 기관-위치 정보 맵 생성
inst_location_map = df_matched.groupby('organization_cleaned')[['city', 'country']].first().to_dict('index')
# 'matched' 데이터에서 논문별(pubid) 소속 기관 정보 생성
pub_org_map = df_matched.groupby('pubid')['organization_cleaned'].unique().apply(list).to_dict()

# 3-2. (증거 수집) 각 'rest' 레코드에 대한 후보 기관 수집
candidate_orgs = []
for idx, row in df_rest.iterrows():
    pubid = row['pubid']
    city = row['city']
    country = row['country']
    
    candidates = []
    
    # [증거 1: 공동 저자] 같은 논문의 'matched' 저자 소속 기관을 후보로 추가
    if pubid in pub_org_map:
        candidates.extend(pub_org_map[pubid])
        
    # [증거 2: 지리 정보] 해당 논문의 모든 연관 기관 중, 위치가 일치하는 기관을 후보로 추가
    # inst 테이블에는 한 pubid에 여러 기관이 있을 수 있음
    possible_orgs = inst[inst['pubid'] == pubid]['organization_cleaned'].unique()
    for org in possible_orgs:
        if org in inst_location_map:
            loc = inst_location_map[org]
            if loc['city'] == city and loc['country'] == country:
                candidates.append(org)

    candidate_orgs.append(candidates)

df_rest['candidate_orgs'] = candidate_orgs

# --- 하이퍼파라미터 설정 ---
# '핵심 저자'로 판단할 최소 논문 수
FREQUENCY_THRESHOLD = 5
# '핵심 저자'의 기관을 결정하기 위한 최소 신뢰도(증거 일치율)
HIGH_CONFIDENCE_THRESHOLD = 0.8  # 80%

# 3-3-1. 저자별 논문 수를 계산하여 '핵심 저자' 그룹을 정의
author_counts = df['full_name'].value_counts()
frequent_authors = set(author_counts[author_counts >= FREQUENCY_THRESHOLD].index)

# 3-3-2. 저자 프로필 생성 (기존과 동일)
author_profiles = {}
# 'matched' 데이터로 프로필 초기화
for _, row in df_matched.iterrows():
    author = row['full_name']
    org = row['organization_cleaned']
    if author not in author_profiles:
        author_profiles[author] = []
    author_profiles[author].append(org)

# 'rest' 데이터의 후보 기관들을 프로필에 추가
for _, row in df_rest.iterrows():
    author = row['full_name']
    if author not in author_profiles:
        author_profiles[author] = []
    author_profiles[author].extend(row['candidate_orgs'])

# 3-3-3. 저자별로 중요도에 따라 다르게 'inferred_org' 결정
inferred_org_map = {}
for author, org_list in author_profiles.items():
    if not org_list:
        continue

    counts = Counter(org_list)
    most_common_org, top_count = counts.most_common(1)[0]
    
    # [Safe Mode] 저자가 '핵심 저자' 그룹에 속하는 경우
    if author in frequent_authors:
        total_evidence = len(org_list)
        confidence = top_count / total_evidence
        
        # 신뢰도가 설정된 임계값을 넘을 때만 기관을 할당
        if confidence >= HIGH_CONFIDENCE_THRESHOLD:
            inferred_org_map[author] = most_common_org
        # 넘지 못하면 실수를 피하기 위해 할당하지 않음 (결과적으로 NaN)
            
    # [Standard Mode] 저자가 '일반 저자' 그룹에 속하는 경우
    else:
        # 기존 방식대로 가장 빈도가 높은 기관을 할당
        inferred_org_map[author] = most_common_org

# 3-4-1. 추론된 주 기관명을 먼저 할당합니다.
df_rest['organization_cleaned'] = df_rest['full_name'].map(inferred_org_map)

# 3-4-2. 기관명에 맞는 city, country를 매핑하기 위한 map을 준비합니다.
# inst_location_map에서 city와 country를 위한 별도의 딕셔너리를 생성합니다.
city_map = {org: loc['city'] for org, loc in inst_location_map.items()}
country_map = {org: loc['country'] for org, loc in inst_location_map.items()}

# 3-4-3. city와 country 정보를 업데이트합니다.
# 추론된 organization_cleaned를 기준으로 새로운 위치 정보를 매핑합니다.
# 만약 매핑되는 정보가 없다면(NaN), 기존의 위치 정보를 그대로 사용(fillna)합니다.
df_rest['city'] = df_rest['organization_cleaned'].map(city_map).fillna(df_rest['city'])
df_rest['country'] = df_rest['organization_cleaned'].map(country_map).fillna(df_rest['country'])

# 3-4-4. suborganization은 여전히 알 수 없으므로 NaN 처리합니다.
df_rest['suborganization_cleaned'] = np.nan 

# 추론에 사용된 임시 컬럼 삭제
df_rest = df_rest.drop(columns=['candidate_orgs'])


# 4. 보호된 그룹과 처리된 그룹 다시 결합 (기존과 유사)
# df_rest에서 기관 정보가 없는(추론 실패한) 경우는 제외하거나 포함할 수 있음. 여기서는 포함.
df_final = pd.concat([df_matched, df_rest], ignore_index=True)


# 5. ID 통일 로직 실행 (기존과 동일)
# ... (이하 ID 통일 및 저장 코드는 모두 동일합니다) ...

# Step 1. 그룹별 canonical ID 찾기
min_ids = df_final.groupby(['full_name', 'organization_cleaned'])['author_id_te'].min()
matched_ids = df_final[df_final['matched'] == 'matched'].groupby(['full_name', 'organization_cleaned'])['author_id_te'].first()
canonical_s = matched_ids.combine_first(min_ids)
canonical_ids = canonical_s.reset_index(name='canonical_id')
canonical_ids['canonical_id'] = canonical_ids['canonical_id'].astype(df_final['author_id_te'].dtype)

# Step 2. df에 canonical ID 병합
df_final = df_final.merge(canonical_ids, on=['full_name','organization_cleaned'], how='left')

# Step 3. 'matched' ID를 최우선으로 하여 ID 전파
final_id_from_matched = df_final[df_final['matched'] == 'matched'].groupby('full_name')['canonical_id'].first()
default_final_id = df_final.groupby('full_name')['canonical_id'].min()
final_id_map = final_id_from_matched.combine_first(default_final_id)
df_final['author_id_te_cleaned'] = df_final['full_name'].map(final_id_map)

# Step 4. cleaned 여부 표시
df_final['cleaned_or_not'] = np.where(
    df_final['author_id_te'] == df_final['author_id_te_cleaned'],
    'original',
    'cleaned'
)

# 6. 최종 정리 및 저장 (기존과 동일)
df_final = df_final.drop(columns=['canonical_id'])
df_final = df_final.drop_duplicates(subset=['author_id_te_cleaned', 'pubid', 'organization_cleaned'])

df_final.to_parquet(dir + 'quant_author_ed_eu_val_cleaned_inferred.parquet')

print("데이터 정제 및 저장이 완료되었습니다.")
print("추론 후 데이터 샘플:")
print(df_final[df_final['matched'] == 'rest'].head())

### Old version 2

In [ ]:
import pandas as pd
import numpy as np

dir = 'D:/GD_awekimm/[YU]/[Project]/[Quantum]/Quantum_2nd/04_Analysis/QuanTech_R2/R file/'

# 1. 데이터 로딩
df = pd.read_csv(dir+'quant_author_ed_eu_val.csv')
inst = pd.read_csv(dir+'quant_inst_ed_eu_val_cleaned.csv')

# ⭐️ (핵심) 원본 파일에 '정답' 기관 정보가 반드시 포함되어 있어야 합니다.
# 만약 실제 컬럼 이름이 다르다면 이 부분을 꼭 수정해주세요.
required_columns = [
    'pubid', 'city', 'country', 'pubyear', 'author_id_te', 
    'full_name', 'matched', 'organization_cleaned', 'suborganization_cleaned'
]
df = df[required_columns]
inst = inst[['pubid', 'organization_cleaned', 'suborganization_cleaned']]

df = df.drop_duplicates()
df['full_name'] = df['full_name'].str.replace(r'\.$', '', regex=True)


# 2. '정답' 그룹과 '처리 대상' 그룹 분리
df_matched = df[df['matched'] == 'matched'].copy()
df_rest = df[df['matched'] == 'rest'].copy()


# 3. 각 그룹 처리
# 3-1. (보호) 'matched' 그룹은 '정답'이므로 아무것도 하지 않고 그대로 사용합니다.
#       inst 파일과 병합하지 않으므로 원본 정보가 100% 보존됩니다.

# 3-2. (처리) 'rest' 그룹은 기관 정보가 불확실하므로 inst 파일과 병합합니다.
#       병합 전, 기존의 (불확실한) 기관 컬럼을 삭제하여 충돌을 방지합니다.
df_rest = df_rest.drop(columns=['organization_cleaned', 'suborganization_cleaned'])
df_rest = pd.merge(df_rest, inst, on='pubid', how='left')
del inst


# 4. 보호된 그룹과 처리된 그룹 다시 결합
df_final = pd.concat([df_matched, df_rest], ignore_index=True)


# 5. ID 통일 로직 실행 (이제 안전하게 조합된 데이터를 대상으로 실행)
# Step 1. 그룹별 canonical ID 찾기
min_ids = df_final.groupby(['full_name', 'organization_cleaned'])['author_id_te'].min()
matched_ids = df_final[df_final['matched'] == 'matched'].groupby(['full_name', 'organization_cleaned'])['author_id_te'].first()
canonical_s = matched_ids.combine_first(min_ids)
canonical_ids = canonical_s.reset_index(name='canonical_id')
canonical_ids['canonical_id'] = canonical_ids['canonical_id'].astype(df_final['author_id_te'].dtype)

# Step 2. df에 canonical ID 병합
df_final = df_final.merge(canonical_ids, on=['full_name','organization_cleaned'], how='left')

# Step 3. 'matched' ID를 최우선으로 하여 ID 전파
final_id_from_matched = df_final[df_final['matched'] == 'matched'].groupby('full_name')['canonical_id'].first()
default_final_id = df_final.groupby('full_name')['canonical_id'].min()
final_id_map = final_id_from_matched.combine_first(default_final_id)
df_final['author_id_te_cleaned'] = df_final['full_name'].map(final_id_map)

# Step 4. cleaned 여부 표시
df_final['cleaned_or_not'] = np.where(
    df_final['author_id_te'] == df_final['author_id_te_cleaned'],
    'original',
    'cleaned'
)

# 6. 최종 정리 및 저장
df_final = df_final.drop(columns=['canonical_id'])
# 중복 제거 기준을 더 명확히 하여 데이터 안정성 확보
df_final = df_final.drop_duplicates(subset=['author_id_te_cleaned', 'pubid', 'organization_cleaned'])

# 경로에 특수문자('[', ']')가 없는지 다시 한번 확인해주세요.
df_final.to_parquet(dir + 'quant_author_ed_eu_val_cleaned.parquet')

print("데이터 정제 및 저장이 완료되었습니다.")
print(df_final.head())

### Old version 3